In [1]:
# import requests
# import zipfile
# import io
import schedule
import time
import pandas as pd
import numpy as np
from datetime import datetime, timezone

In [2]:
# url = "https://s3.amazonaws.com/dl.ncsbe.gov/data/ncvoter90.zip"
# r = requests.get(url)
# r.ok

# z = zipfile.ZipFile(io.BytesIO(r.content))
# z.extractall("App_Data/")

# newest_vreg = pd.read_table("App_Data/ncvoter90.txt", encoding='ISO-8859-1')
# newest_vreg.head()

In [3]:
url = "https://s3.amazonaws.com/dl.ncsbe.gov/data/ncvoter90.zip"

# newest_vreg = pd.read_table(url, encoding='ISO-8859-1')
# newest_vreg.head()

In [4]:
newest_vreg['ncid'].nunique()

NameError: name 'newest_vreg' is not defined

In [5]:
def get_birth_reg_census(state):
    
    if state in ['AS', 'GU', 'MP', 'PR', 'VI', 'OC']:
        return 'Other'
    
    # the rest of the categories are based on U.S. Census Bureau regions
    elif state in ['CT', 'ME', 'MA', 'NH', 'RI', 'VT',
                     'NJ', 'NY', 'PA']:
        return 'Northeast'
    
    elif state in ['DE', 'FL', 'GA', 'MD', 'NC', 'SC', 'VA', 
                     'DC', 'WV', 'AL', 'KY', 'MS', 'TN', 'AR',
                     'LA', 'OK', 'TX']:
        return 'South'
    
    elif state in ['IL', 'IN', 'MI', 'OH', 'WI',
                     'IA', 'KS', 'MN', 'MO', 'NE', 'ND', 'SD']:
        return 'Midwest'
    
    elif state in ['AZ', 'CO', 'ID', 'MT', 'NV', 'NM', 'UT',
                     'WY', 'AK', 'CA', 'HI', 'OR', 'WA']:
        return 'West'
    
    else:
        return 'Missing'

In [6]:
def get_gen_grp(birth_year):
    
    if birth_year < 1946:
        return 'Greatest-Silent'
    
    elif (birth_year > 1945) & (birth_year < 1965):
        return 'Boomer'
    
    elif (birth_year > 1964) & (birth_year < 1981):
        return 'GenX'
    
    elif (birth_year > 1980) & (birth_year < 1997):
        return 'Millennial'
    
    elif birth_year > 1996:
        return 'GenZ'
    
    else:
        return 'Missing'

In [7]:
def clean_vreg(df):
    ## Recast registr_dt as datetime variable
    df['registr_dt'] = pd.to_datetime(df['registr_dt'])

    ## Fill null values in birth_state with 'Missing'
    df['birth_state'].fillna(value='Missing', inplace=True)

    # Recast drivers_lic for sake of clarity in figures
    df['drivers_lic'] = np.where(df['drivers_lic']=='Y',
                                 'License',
                                 'No License')

    ## Create new column grouping most infrequent party categories (<5% of voters)
      ## into same group as those who are unaffiliated
    df['party_grp'] = np.where(df['party_cd'].isin(['REP', 'DEM']),
                               df['party_cd'].str.title(),
                               'Other')

    ## Create new column grouping most infrequent race categories (<5% of voters)
    df['race_grp'] = np.where(df['race_code'].isin(['W', 'B', 'U']),
                              df['race_code'],
                              'O')
    
    race_grp_map = {'W': 'White',
                    'B': 'Black',
                    'U': 'Undesig.',
                    'O': 'Other'}
    
    df['race_grp'] = df['race_grp'].map(race_grp_map)

    ## Create new column grouping most infrequent cities (<5% of voters)
    df['res_city_desc'].fillna('Missing', inplace=True)
    df['city_grp'] = np.where(df['res_city_desc'].isin(['MONROE', 
                                                        'WAXHAW',
                                                        'INDIAN TRAIL',
                                                        'MATTHEWS',
                                                        'Missing']),
                              df['res_city_desc'].str.title(),
                              'Other')

    ## Create a new column grouping birth_state into U.S. Census regions,
      ## lumping territories and out of country into 'Other'
    df['birth_reg_other'] = df['birth_state'].apply(get_birth_reg_census)

    ## Create a new column grouping birth_year into generations, 
      ## also lumping Silent in with Greatest
    df['gen_grp'] = df['birth_year'].apply(get_gen_grp)

    ## Reformat voter_status_desc labels
    df['voter_status_desc'] = np.where(
        df['voter_status_desc']=='TEMPORARY',
        'Temp',
        df['voter_status_desc'].str.title())

    ## Select only the necessary columns
    cleaned_df = df[['voter_status_desc', 'reason_cd', 'city_grp', 
             'race_grp', 'party_grp', 'gen_grp', 'gender_code', 
             'birth_age', 'birth_reg_other', 'drivers_lic',
             'registr_dt']].copy()
    
    return cleaned_df

In [29]:
url = "https://s3.amazonaws.com/dl.ncsbe.gov/data/ncvoter90.zip"

class VregData:
    
    def __init__(self, url):
        self.url = url
        self.raw_data = pd.read_table(url, encoding='ISO-8859-1')
        self.dt_retrieved = datetime.now(timezone.utc).astimezone().strftime("%m/%d/%Y %H:%M:%S %Z")
        
    def clean_data(self):
        self.clean_df = clean_vreg(self.raw_data)
    
    def sched_retrieval(self, url):
        schedule.every(20).seconds.do(self.__init__, url)
        while 1:
            schedule.run_pending()
            time.sleep(1)

In [30]:
newest_vreg = VregData(url)

In [31]:
newest_vreg.dt_retrieved

'02/17/2021 13:17:13 EST'

In [26]:
newest_vreg.sched_retrieval(url)

Retrieved at 02/17/2021 12:54:46 EST


<bound method NDFrame.head of        voter_status_desc reason_cd      city_grp  race_grp party_grp  \
0                 Active        AV        Waxhaw     White     Other   
1                 Active        AV        Waxhaw     Other       Dem   
2               Inactive        IN  Indian Trail     White     Other   
3                 Active        AV  Indian Trail     White     Other   
4                 Active        AV  Indian Trail     White       Rep   
...                  ...       ...           ...       ...       ...   
179670            Active        AV  Indian Trail     White       Rep   
179671            Active        AV        Monroe  Undesig.       Rep   
179672           Removed        RL       Missing     White       Rep   
179673            Active        AV        Monroe     White       Dem   
179674          Inactive        IU      Matthews     White       Dem   

           gen_grp gender_code  birth_age birth_reg_other drivers_lic  \
0             GenX           M  

Retrieved at 02/17/2021 12:54:51 EST


<bound method NDFrame.head of        voter_status_desc reason_cd      city_grp  race_grp party_grp  \
0                 Active        AV        Waxhaw     White     Other   
1                 Active        AV        Waxhaw     Other       Dem   
2               Inactive        IN  Indian Trail     White     Other   
3                 Active        AV  Indian Trail     White     Other   
4                 Active        AV  Indian Trail     White       Rep   
...                  ...       ...           ...       ...       ...   
179670            Active        AV  Indian Trail     White       Rep   
179671            Active        AV        Monroe  Undesig.       Rep   
179672           Removed        RL       Missing     White       Rep   
179673            Active        AV        Monroe     White       Dem   
179674          Inactive        IU      Matthews     White       Dem   

           gen_grp gender_code  birth_age birth_reg_other drivers_lic  \
0             GenX           M  

Retrieved at 02/17/2021 12:54:56 EST


<bound method NDFrame.head of        voter_status_desc reason_cd      city_grp  race_grp party_grp  \
0                 Active        AV        Waxhaw     White     Other   
1                 Active        AV        Waxhaw     Other       Dem   
2               Inactive        IN  Indian Trail     White     Other   
3                 Active        AV  Indian Trail     White     Other   
4                 Active        AV  Indian Trail     White       Rep   
...                  ...       ...           ...       ...       ...   
179670            Active        AV  Indian Trail     White       Rep   
179671            Active        AV        Monroe  Undesig.       Rep   
179672           Removed        RL       Missing     White       Rep   
179673            Active        AV        Monroe     White       Dem   
179674          Inactive        IU      Matthews     White       Dem   

           gen_grp gender_code  birth_age birth_reg_other drivers_lic  \
0             GenX           M  

KeyboardInterrupt: 

In [32]:
newest_vreg.raw_data

,county_id,county_desc,voter_reg_num,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,absent_ind,name_prefx_cd,last_name,...,munic_dist_desc,dist_1_abbrv,dist_1_desc,dist_2_abbrv,dist_2_desc,confidential_ind,birth_year,ncid,vtd_abbrv,vtd_desc
0,90,UNION,346446,A,ACTIVE,AV,VERIFIED,,,AALAEI,...,WAXHAW,30.0,30TH PROSECUTORIAL,,,N,1974,EF230133,020A,020A
1,90,UNION,348004,A,ACTIVE,AV,VERIFIED,,,AALAEI,...,WAXHAW,30.0,30TH PROSECUTORIAL,,,N,1974,EF231096,020A,020A
2,90,UNION,281016,I,INACTIVE,IN,CONFIRMATION NOT RETURNED,,,AALBORG,...,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1994,AL261840,029A,029A
3,90,UNION,288489,A,ACTIVE,AV,VERIFIED,,,AALBORG,...,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1958,AL178177,029A,029A
4,90,UNION,279308,A,ACTIVE,AV,VERIFIED,,,AALBORG,...,INDIAN TRAIL,30.0,30TH PROSECUTORIAL,,,N,1958,AL193958,029A,029A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179670,90,UNION,336361,A,ACTIVE,AV,VERIFIED,,,ZYCZKIEWICZ,...,HEMBY BRIDGE,30.0,30TH PROSECUTORIAL,,,N,1989,BL403374,016,016
179671,90,UNION,352857,A,ACTIVE,AV,VERIFIED,,,ZYGNER,...,MONROE,30.0,30TH PROSECUTORIAL,,,N,1990,AN145460,007,007
179672,90,UNION,301279,R,REMOVED,RL,MOVED FROM COUNTY,,,ZYKA,...,NaN,NaN,NaN,,,N,1995,EF199981,NaN,NaN
179673,90,UNION,243464,A,ACTIVE,AV,VERIFIED,,,ZYNSKY,...,NaN,30.0,30TH PROSECUTORIAL,,,N,1952,EF157599,025,025


In [33]:
newest_vreg.clean_data()

,voter_status_desc,reason_cd,city_grp,race_grp,party_grp,gen_grp,gender_code,birth_age,birth_reg_other,drivers_lic,registr_dt
0,Active,AV,Waxhaw,White,Other,GenX,M,46,Other,License,2019-04-25
1,Active,AV,Waxhaw,Other,Dem,GenX,F,46,Other,License,2019-06-24
2,Inactive,IN,Indian Trail,White,Other,Millennial,F,27,Midwest,License,2013-11-04
3,Active,AV,Indian Trail,White,Other,Boomer,M,62,South,License,2014-09-17
4,Active,AV,Indian Trail,White,Rep,Boomer,F,62,Northeast,License,2013-07-24
...,...,...,...,...,...,...,...,...,...,...,...
179670,Active,AV,Indian Trail,White,Rep,Millennial,M,31,Northeast,License,2018-10-04
179671,Active,AV,Monroe,Undesig.,Rep,Millennial,U,30,Missing,License,2019-10-09
179672,Removed,RL,Missing,White,Rep,Millennial,U,25,Missing,No License,2016-02-18
179673,Active,AV,Monroe,White,Dem,Boomer,M,68,Northeast,License,2009-08-28


In [13]:
newest_vreg.clean_df

,voter_status_desc,reason_cd,city_grp,race_grp,party_grp,gen_grp,gender_code,birth_age,birth_reg_other,drivers_lic,registr_dt
0,Active,AV,Waxhaw,White,Other,GenX,M,46,Other,License,2019-04-25
1,Active,AV,Waxhaw,Other,Dem,GenX,F,46,Other,License,2019-06-24
2,Inactive,IN,Indian Trail,White,Other,Millennial,F,27,Midwest,License,2013-11-04
3,Active,AV,Indian Trail,White,Other,Boomer,M,62,South,License,2014-09-17
4,Active,AV,Indian Trail,White,Rep,Boomer,F,62,Northeast,License,2013-07-24
...,...,...,...,...,...,...,...,...,...,...,...
179670,Active,AV,Indian Trail,White,Rep,Millennial,M,31,Northeast,License,2018-10-04
179671,Active,AV,Monroe,Undesig.,Rep,Millennial,U,30,Missing,License,2019-10-09
179672,Removed,RL,Missing,White,Rep,Millennial,U,25,Missing,No License,2016-02-18
179673,Active,AV,Monroe,White,Dem,Boomer,M,68,Northeast,License,2009-08-28


In [14]:
# cleaned_vreg = clean_vreg(newest_vreg)
# cleaned_vreg.head()

In [15]:
# cleaned_vreg.info()

In [18]:
def test_new_df(url):
    
    new_data = VregData(url)
    new_data.clean_data()
    new_df = new_data.clean_df
    
    # datetime object containing current date and time
    print("Retrieved at {}".format(new_data.dt_retrieved))
    
    display(new_df.head)
    
schedule.every(20).seconds.do(test_new_df, url)

while 1:
    schedule.run_pending()
    time.sleep(1)

Retrieved at 02/17/2021 12:39:59 EST


<bound method NDFrame.head of        voter_status_desc reason_cd      city_grp  race_grp party_grp  \
0                 Active        AV        Waxhaw     White     Other   
1                 Active        AV        Waxhaw     Other       Dem   
2               Inactive        IN  Indian Trail     White     Other   
3                 Active        AV  Indian Trail     White     Other   
4                 Active        AV  Indian Trail     White       Rep   
...                  ...       ...           ...       ...       ...   
179670            Active        AV  Indian Trail     White       Rep   
179671            Active        AV        Monroe  Undesig.       Rep   
179672           Removed        RL       Missing     White       Rep   
179673            Active        AV        Monroe     White       Dem   
179674          Inactive        IU      Matthews     White       Dem   

           gen_grp gender_code  birth_age birth_reg_other drivers_lic  \
0             GenX           M  

Retrieved at 02/17/2021 12:40:05 EST


<bound method NDFrame.head of        voter_status_desc reason_cd      city_grp  race_grp party_grp  \
0                 Active        AV        Waxhaw     White     Other   
1                 Active        AV        Waxhaw     Other       Dem   
2               Inactive        IN  Indian Trail     White     Other   
3                 Active        AV  Indian Trail     White     Other   
4                 Active        AV  Indian Trail     White       Rep   
...                  ...       ...           ...       ...       ...   
179670            Active        AV  Indian Trail     White       Rep   
179671            Active        AV        Monroe  Undesig.       Rep   
179672           Removed        RL       Missing     White       Rep   
179673            Active        AV        Monroe     White       Dem   
179674          Inactive        IU      Matthews     White       Dem   

           gen_grp gender_code  birth_age birth_reg_other drivers_lic  \
0             GenX           M  

KeyboardInterrupt: 

In [ ]:
def get_new_df(url):
    
    new_data = VregData(url)
    new_data.clean_data()
    new_df = new_data.clean_df
    
    return(new_df)
    
schedule.every(20).seconds.do(get_new_df, url)

while 1:
    schedule.run_pending()
    time.sleep(1)

In [37]:
datetime.today().weekday()

2

In [40]:
datetime.now(timezone.utc).strftime("%m/%d/%Y")

'02/17/2021'